In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import torch
import copy
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import Tensor

In [2]:
train = pd.read_csv('dataAll.csv')
test = pd.read_csv('testAll.csv')
train_data=train.iloc[:,:30].applymap(lambda x: x+126.23).apply(lambda x: (x - x.mean()) 
                                                                / (x.std()))
train_loc = train.iloc[:,30:]

In [3]:
#all_features[numeric_feats] = all_features[numeric_feats].apply(lambda x: (x - x.mean()) 
                                                              #  / (x.std()))

In [4]:
train_feat = torch.from_numpy(
        train_data.values.astype(np.float32))
train_labels = torch.from_numpy(
        train_loc.values.astype(np.float32))

In [5]:
num_train = int(0.8 * train_feat.shape[0]) # 划分训练样本和验证集样本
indices = np.arange( train_feat.shape[0])
np.random.shuffle(indices)  # shuffle 顺序
train_indices = indices[:num_train]
valid_indices = indices[num_train:]

train_features = train_feat[train_indices,:]
train_label = train_labels[train_indices,:]
valid_features  = train_feat[valid_indices,:]
valid_labels = train_labels[valid_indices,:]


In [6]:
feat_dim = train_data.shape[0]

In [9]:
def error(y_test,y_pre):
    y_test = np.array(y_test)
    y_pre = np.array(y_pre)
    m = y_test-y_pre
    m=np.square(m)
    m=m.sum(axis=1)
    m=np.sqrt(m)
    return m.sum()/len(y_test)

def get_model(feat_dim,output_dim = 2 ):
    net = nn.Sequential(
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim*4/5)),
        nn.BatchNorm1d(int(feat_dim*4/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*4/5),int(feat_dim*3/5)),
        nn.BatchNorm1d(int(feat_dim*3/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*3/5),int(feat_dim*2/5)),
        nn.BatchNorm1d(int(feat_dim*2/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*2/5),int(feat_dim/5)),
        nn.BatchNorm1d(int(feat_dim/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim/5), output_dim)


        )
    return net

def get_data(x,y,batch_size,shuffle):
    dataset = TensorDataset(x,y)
    return DataLoader(dataset, batch_size, shuffle=shuffle,num_workers=4)

def train_model(model,x_train,y_train,x_valid,y_valid,epochs,batch_size,lr,weight_decay,use_gpu):
    
    if use_gpu:
        model = model.cuda()
    train_data = get_data(x_train,y_train,batch_size,True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    #criterion = nn.CrossEntropyLoss()
    best_error = 0
    for e in  range(epochs):
        model.train()
        for data in train_data:
            x,y = data
            if use_gpu:
                x = x.cuda()
                y = y.cuda()
                
            out = model(x)
            loss = criterion(out, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if use_gpu:
                x_valid = x_valid.cuda()
        with torch.no_grad():
            output = model(x_valid)
        output = Tensor.cpu(output)
        out = output.numpy()
        print('epoch:{}'.format(e))
        error_i = error(out,y_valid)
        print('error:{}'.format(error_i)) 
        if e == 0:
            best_error = error_i
        elif error_i < best_error:
            best_model = copy.deepcopy(model)
    torch.save(best_model,"model.t7")

def predict(x_test):
    net = torch.load('./model.t7')
    a = net(x_test).detach().numpy()
    a = np.argmax(a,axis=1)
    with open('./ans.txt','w') as f:
        for i in a:
            f.write('{}'.format(i))
            f.write('\n')
    

In [10]:
# 可以调整的超参
batch_size = 128
epochs = 20000
lr = 0.001
wd = 0
use_gpu = True
feat_dim = train_feat.shape[1]
net = get_model(feat_dim)
train_model(net, train_features, train_label, valid_features, valid_labels, epochs, 
            batch_size, lr, wd, use_gpu)
#predict(x_test)

epoch:0
error:37.32581055950841
epoch:1
error:37.21496503072445
epoch:2
error:37.116667003557566
epoch:3
error:37.01000060640362
epoch:4
error:36.89060478654592
epoch:5
error:36.76743915750323
epoch:6
error:36.626402308376456
epoch:7
error:36.47122109476067
epoch:8
error:36.311026944534284
epoch:9
error:36.16445918903622
epoch:10
error:35.98563328751617
epoch:11
error:35.78095437823415
epoch:12
error:35.53455995310479
epoch:13
error:35.29608920197283
epoch:14
error:35.02949900954075
epoch:15
error:34.734410373544634
epoch:16
error:34.4561418580207
epoch:17
error:34.154890645213456
epoch:18
error:33.82691724611902
epoch:19
error:33.50447475339586
epoch:20
error:33.14242399741268
epoch:21
error:32.81176726228978
epoch:22
error:32.4731540063066
epoch:23
error:32.09240580530401
epoch:24
error:31.717276944534284
epoch:25
error:31.387309993531694
epoch:26
error:30.981625970245794
epoch:27
error:30.506455671895214
epoch:28
error:30.0628234152652
epoch:29
error:29.676044024902975
epoch:30
erro

epoch:241
error:2.12955750222348
epoch:242
error:2.201105865439036
epoch:243
error:2.249499401176423
epoch:244
error:2.1464744261905726
epoch:245
error:2.1995342693847024
epoch:246
error:2.2825844038041723
epoch:247
error:2.352188485405886
epoch:248
error:2.1379214505174646
epoch:249
error:2.3161883236982534
epoch:250
error:2.2578068149660413
epoch:251
error:2.30444225395173
epoch:252
error:2.260714394000647
epoch:253
error:2.2044982515362226
epoch:254
error:2.119149152550938
epoch:255
error:2.121757951467497
epoch:256
error:2.086230753001698
epoch:257
error:2.1246311044631305
epoch:258
error:2.1609676306799805
epoch:259
error:2.1396020097226716
epoch:260
error:2.3520841018657017
epoch:261
error:2.183923481969599
epoch:262
error:2.2854558197060966
epoch:263
error:2.1295098111052715
epoch:264
error:2.1058038511683375
epoch:265
error:2.0852268707551747
epoch:266
error:2.0609284039456663
epoch:267
error:2.0597234925816625
epoch:268
error:2.050196323172704
epoch:269
error:2.106151585745472

epoch:478
error:1.7972018894526196
epoch:479
error:1.7635472780057406
epoch:480
error:1.7784870734961191
epoch:481
error:1.7635444354887613
epoch:482
error:1.7375864125161709
epoch:483
error:1.816137790063066
epoch:484
error:1.8261466081824063
epoch:485
error:1.7652346276681758
epoch:486
error:1.8137905027086028
epoch:487
error:1.7124868296713291
epoch:488
error:1.7797829454034606
epoch:489
error:1.8142882590152005
epoch:490
error:1.7292482047926099
epoch:491
error:1.7734608718062743
epoch:492
error:1.7124693008166236
epoch:493
error:1.743375198344518
epoch:494
error:1.822012798906452
epoch:495
error:1.822058121260511
epoch:496
error:1.7577275403258408
epoch:497
error:1.680342384328509
epoch:498
error:1.6528683523002912
epoch:499
error:1.6760351815168175
epoch:500
error:1.7229760331601713
epoch:501
error:1.687235645920925
epoch:502
error:1.6676300167266334
epoch:503
error:1.7905503997210543
epoch:504
error:1.8456142173855918
epoch:505
error:1.7889964904390363
epoch:506
error:1.67715987

epoch:714
error:1.6219673502183054
epoch:715
error:1.6183631966061611
epoch:716
error:1.6281507721539457
epoch:717
error:1.5369805142302717
epoch:718
error:1.5532352896587969
epoch:719
error:1.547994004184185
epoch:720
error:1.5563101034423512
epoch:721
error:1.4947325002021346
epoch:722
error:1.6686220551524094
epoch:723
error:1.6032237932567917
epoch:724
error:1.5516599035818242
epoch:725
error:1.6208350809548835
epoch:726
error:1.5336045515645214
epoch:727
error:1.5273401178949708
epoch:728
error:1.5911348843790427
epoch:729
error:1.5539975579620795
epoch:730
error:1.6476952872331825
epoch:731
error:1.5564383325416398
epoch:732
error:1.5704194102219438
epoch:733
error:1.6850598570908797
epoch:734
error:1.6652198781634056
epoch:735
error:1.4997286975460866
epoch:736
error:1.528430223156533
epoch:737
error:1.5306934983424967
epoch:738
error:1.6890106398568887
epoch:739
error:1.6114651974854464
epoch:740
error:1.640608576568564
epoch:741
error:1.7241998946373707
epoch:742
error:1.62261

epoch:950
error:1.5542416985870795
epoch:951
error:1.5634572965515847
epoch:952
error:1.5531116401701972
epoch:953
error:1.5768101779794632
epoch:954
error:1.6463798335422057
epoch:955
error:1.5278790906977684
epoch:956
error:1.5981113685519082
epoch:957
error:1.5198275034362871
epoch:958
error:1.559098138846216
epoch:959
error:1.9217884611497413
epoch:960
error:1.7449156846296896
epoch:961
error:1.7864060099652328
epoch:962
error:1.6957082415305627
epoch:963
error:1.7297758075274903
epoch:964
error:1.8182987346377748
epoch:965
error:1.6784039456662354
epoch:966
error:1.6416350410333118
epoch:967
error:1.6084524453226068
epoch:968
error:1.5728977691926747
epoch:969
error:1.5497284132943887
epoch:970
error:1.5850474763502587
epoch:971
error:1.5725699322343951
epoch:972
error:1.523856297501617
epoch:973
error:1.5217351481646184
epoch:974
error:1.564102863741106
epoch:975
error:1.6124379699628073
epoch:976
error:1.5569855170601552
epoch:977
error:1.564204562681921
epoch:978
error:1.492515

...

epoch:19078
error:1.2295637241773125
epoch:19079
error:1.2414721332268759
epoch:19080
error:1.2971858608152087
epoch:19081
error:1.2519352803100743
epoch:19082
error:1.2477318293681274
epoch:19083
error:1.289880197384379
epoch:19084
error:1.2660172673431436
epoch:19085
error:1.244766926241106
epoch:19086
error:1.2768889472833118
epoch:19087
error:1.2630111477199224
epoch:19088
error:1.2395739635551424
epoch:19089
error:1.2122876955651682
epoch:19090
error:1.2343982928474693
epoch:19091
error:1.2235503005487953
epoch:19092
error:1.2288580693372009
epoch:19093
error:1.2456796900266818
epoch:19094
error:1.2097150597812905
epoch:19095
error:1.2104844343770214
epoch:19096
error:1.3174292402773287
epoch:19097
error:1.2871165602512533
epoch:19098
error:1.2388091685701001
epoch:19099
error:1.2232494675018193
epoch:19100
error:1.2331880912435318
epoch:19101
error:1.2048237197303526
epoch:19102
error:1.2064054225117238
epoch:19103
error:1.228088142029835
epoch:19104
error:1.2059763603654592
epoc

epoch:19301
error:1.3438057449163163
epoch:19302
error:1.2747101580186773
epoch:19303
error:1.2598352666659929
epoch:19304
error:1.2574065727835948
epoch:19305
error:1.257767098687136
epoch:19306
error:1.2946905257014067
epoch:19307
error:1.243282184872251
epoch:19308
error:1.2335219290709896
epoch:19309
error:1.2434386022649175
epoch:19310
error:1.2620375066957066
epoch:19311
error:1.2692367336574224
epoch:19312
error:1.233023935887977
epoch:19313
error:1.215076362639473
epoch:19314
error:1.2343600767858587
epoch:19315
error:1.2496026003345326
epoch:19316
error:1.2996391898447606
epoch:19317
error:1.276721791493168
epoch:19318
error:1.2501003566411304
epoch:19319
error:1.2313455085705045
epoch:19320
error:1.2661183346135187
epoch:19321
error:1.2568630203701083
epoch:19322
error:1.2386908882802394
epoch:19323
error:1.2507710327306356
epoch:19324
error:1.2573509068260835
epoch:19325
error:1.2197895716263745
epoch:19326
error:1.2393989908433054
epoch:19327
error:1.2162946180415184
epoch:

epoch:19524
error:1.2945548744744502
epoch:19525
error:1.2743073101956663
epoch:19526
error:1.2367903498443564
epoch:19527
error:1.2228787559124352
epoch:19528
error:1.2259017727199224
epoch:19529
error:1.217789939890241
epoch:19530
error:1.2484265089343467
epoch:19531
error:1.2692691067674644
epoch:19532
error:1.2371357946161465
epoch:19533
error:1.2199653339262613
epoch:19534
error:1.2389509785838455
epoch:19535
error:1.2258158655401035
epoch:19536
error:1.2185535504932083
epoch:19537
error:1.2533551965000405
epoch:19538
error:1.2240497149902976
epoch:19539
error:1.2592890296531372
epoch:19540
error:1.2214613664042286
epoch:19541
error:1.243986734289093
epoch:19542
error:1.2553609080641575
epoch:19543
error:1.2264746188500566
epoch:19544
error:1.266101121594033
epoch:19545
error:1.2064706424846379
epoch:19546
error:1.2065639717921248
epoch:19547
error:1.2280530843204236
epoch:19548
error:1.2035490087827458
epoch:19549
error:1.2175078990388501
epoch:19550
error:1.220711336715718
epoch

epoch:19747
error:1.2556241567199629
epoch:19748
error:1.248014501889958
epoch:19749
error:1.230329861462039
epoch:19750
error:1.3358950201629205
epoch:19751
error:1.2400523749545198
epoch:19752
error:1.262504705944777
epoch:19753
error:1.233039964525388
epoch:19754
error:1.275932124484557
epoch:19755
error:1.23569748102462
epoch:19756
error:1.250187527161829
epoch:19757
error:1.2474026501101634
epoch:19758
error:1.2698838010642384
epoch:19759
error:1.2485482634116267
epoch:19760
error:1.3207008194028946
epoch:19761
error:1.2546492523548674
epoch:19762
error:1.2506285120876455
epoch:19763
error:1.248641276883894
epoch:19764
error:1.2296249962099774
epoch:19765
error:1.221895245037597
epoch:19766
error:1.289408497483425
epoch:19767
error:1.233199698187864
epoch:19768
error:1.2297176148882196
epoch:19769
error:1.2059746232717496
epoch:19770
error:1.2485245757701326
epoch:19771
error:1.2279480691097995
epoch:19772
error:1.2532641569978977
epoch:19773
error:1.2458973004932083
epoch:19774
e

epoch:19971
error:1.3041769523669955
epoch:19972
error:1.2655094832683134
epoch:19973
error:1.2726669410221945
epoch:19974
error:1.2751115845831986
epoch:19975
error:1.2626845741025226
epoch:19976
error:1.285559492617036
epoch:19977
error:1.2244555632478977
epoch:19978
error:1.2601102012249352
epoch:19979
error:1.2498552685104705
epoch:19980
error:1.2305212576053122
epoch:19981
error:1.2410951838918984
epoch:19982
error:1.260135547001334
epoch:19983
error:1.2472610769728332
epoch:19984
error:1.2883023635844517
epoch:19985
error:1.3494593532705368
epoch:19986
error:1.3902561044631307
epoch:19987
error:1.2936544282624516
epoch:19988
error:1.2895840229068969
epoch:19989
error:1.37691238225663
epoch:19990
error:1.320247595862306
epoch:19991
error:1.285526487836554
epoch:19992
error:1.2513693825547785
epoch:19993
error:1.3248324178323092
epoch:19994
error:1.2541495220781451
epoch:19995
error:1.2705014957956016
epoch:19996
error:1.2712467089970083
epoch:19997
error:1.2645783220811773
epoch:1

In [ ]:
        
        '''
        nn.Linear(int(feat_dim*4/5),int(feat_dim*3/5)),
        nn.BatchNorm1d(int(feat_dim*3/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*3/5),int(feat_dim*2/5)),
        nn.BatchNorm1d(int(feat_dim*2/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*2/5),int(feat_dim/5)),
        nn.BatchNorm1d(int(feat_dim/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim/5), output_dim)
        '''
        